# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [8]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# 훈련 데이터셋 확인
# base_line에서 train_data -> df_train으로 수정
df_train = pd.read_csv(os.path.join(ROOT_DIR, "train1.csv"))

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 151 entries, Equipment_Dam to target
dtypes: float64(68), int64(62), object(21)
memory usage: 46.7+ MB


In [10]:
train_data.value_counts()

Equipment_Dam     Model.Suffix_Dam  Workorder_Dam  CURE END POSITION X Collect Result_Dam  CURE END POSITION Z Collect Result_Dam  CURE END POSITION Θ Collect Result_Dam  CURE SPEED Collect Result_Dam  CURE START POSITION X Collect Result_Dam  CURE START POSITION Θ Collect Result_Dam  DISCHARGED SPEED OF RESIN Collect Result_Dam  DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam  DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam  DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam  Dispense Volume(Stage1) Collect Result_Dam  Dispense Volume(Stage2) Collect Result_Dam  Dispense Volume(Stage3) Collect Result_Dam  HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam  HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam  HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam  HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam  HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam  HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam  HEAD NORMAL COORDINATE Y AXIS(St

### 결측치 제거

In [11]:
# 1. 결측치 제거 (필요)
# 수치형 데이터에 대해 결측치-> 평균값으로 대체, 비수치형 데이터 결측치 -> 최빈값으로 대체
numeric_columns = df_train.select_dtypes(include=['float64', 'int64']).columns
non_numeric_columns = df_train.select_dtypes(exclude=['float64', 'int64']).columns


df_train[numeric_columns] = df_train[numeric_columns].fillna(df_train[numeric_columns].mean())
for col in non_numeric_columns:
    df_train[col] = df_train[col].fillna(df_train[col].mode()[0])

### 언더 샘플링


- 데이터 불균형을 해결하기 위해 언더 샘플링
- 데이터 불균형을 해결하기 위해 Normal 클래스의 샘플 수를 AbNormal 클래스와 동일하게 조정


In [12]:
# 언더샘플링 (Normal과 AbNormal의 비율을 1:1로 맞춤)

normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

In [16]:
# 피처 선택
features = []
for col in df_concat.columns:
    try:
        df_concat[col] = df_concat[col].astype(int)
        features.append(col)
    except:
        continue


# 타겟 변수와 피처 데이터 분리
train_x = df_concat[features]
train_y = df_concat["target"]

In [17]:
# 라이브러리 load

from sklearn.model_selection import StratifiedKFold, GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

## [ML1]교차 검증 수행

- StratifiedKFold: 데이터를 5개의 폴드로 나누어, 타겟 변수의 비율을 각 폴드에 균등하게 유지
- cross_val_score: 5-Fold 교차 검증을 수행하며, 각 폴드에서의 정확도를 측정


### Validation  

Train : 40506 행  
Test : 17361 행 (Train의 약 40% )



- Train으로 학습하고 Validation으로 검증하고 Test로 최종 성능을 평가
- Train : Validation : Test = 6 : 2 : 2가 적당하지만 Test 데이터가 많음
- Cross Validation 사용



✅ **Cross Validation**
- Training Set을 K-fold 방식으로 쪼개서 모든 데이터를 Training과 Validaion 과정에 사용




In [18]:
# StratifiedKFold 설정 (5-fold)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
# RandomForestClassifier 초기화
model = RandomForestClassifier(random_state=42)

## [ML2] 하이퍼 파라미터 튜닝 수행

- K-Fold 교차 검증을 통해 최적의 하이퍼파라미터를 찾기 위해서는, GridSearchCV 또는 RandomizedSearchCV
-  그리드 서치(Grid Search)나 랜덤 서치(Randomized Search) 두 가지 방식을 모두 시도
- Grid Search는 시간이 너무 오래걸려서 **랜덤 서치만 적용**

In [20]:
# 하이퍼파라미터 범위 설정 (조합 축소)
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}


In [21]:
# RandomizedSearchCV 설정 (StratifiedKFold를 사용하여 교차 검증)
## GridSearchCV는 시간이 너무 오래걸림
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=10, cv=skf, verbose=2, random_state=42,
                                   n_jobs=-1, scoring='accuracy')

# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 탐색
random_search.fit(train_x, train_y)

# 최적의 하이퍼파라미터와 그에 따른 최고 성능 출력
best_params = random_search.best_params_
best_score = random_search.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [22]:
# ✅ Validation Accuracy 확인
print(f"Best Hyperparameters: {best_params}")
print(f"Best Validation Accuracy: {best_score:.4f}")

Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10, 'bootstrap': False}
Best Validation Accuracy: 0.5985


### 모델학습

In [23]:
# 최적의 하이퍼파라미터로 모델 초기화 및 학습
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(train_x, train_y)

RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_leaf=2,
                       n_estimators=200, random_state=42)

In [24]:
# ✅Train Accuracy 확인
# 모델 학습 후 결과 확인 (전체 데이터셋에 대해)
train_accuracy = best_model.score(train_x, train_y)
print(f"Train Accuracy: {train_accuracy:.4f}")

Train Accuracy: 0.7355


### 테스트 데이터 예측


테스트 데이터 불러오기


In [30]:
# 테스트 데이터셋 로드 및 전처리
df_test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 465 entries, Set ID to target
dtypes: float64(351), int64(77), object(37)
memory usage: 61.6+ MB


In [36]:
# 테스트 데이터셋 결측값 처리 (훈련 데이터와 동일한 방식으로 처리)
numeric_columns = df_test.select_dtypes(include=['float64', 'int64']).columns
non_numeric_columns = df_test.select_dtypes(exclude=['float64', 'int64']).columns

df_test[numeric_columns] = df_test[numeric_columns].fillna(df_test[numeric_columns].mean())
for col in non_numeric_columns:
    df_test[col] = df_test[col].fillna(df_test[col].mode()[0])

In [37]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 465 entries, Set ID to target
dtypes: float64(351), int64(77), object(37)
memory usage: 61.6+ MB


In [38]:
# 테스트 데이터셋 피처 선택 (훈련 데이터와 동일한 피처 사용)
test_x = df_test[features]

# 테스트 데이터셋에 대한 예측 및 정확도 평가
test_y_pred = best_model.predict(test_x)
test_accuracy = (test_y_pred == df_test['target']).mean()  # 실제 타겟과 예측 값 비교

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.0000


### 제출 파일 작성


In [39]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_y_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
